In [ ]:
# If needed, run once:
# !pip install pandas numpy scikit-learn plotly openpyxl

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
EXCEL_PATH = "Chelsea_Player_Stats_2025_Clusters-2.xlsx"  # change if needed

def load_position_sheet(path, sheet_name, in_squad_only=False):
    """
    Load FW / MF / DF sheet, keep only weighted-average rows for 2025.
    If in_squad_only=True, keeps only rows with In_Squad / In_squad == 'Yes'.
    """
    df = pd.read_excel(path, sheet_name=sheet_name)
    df = df.dropna(how="all")
    
    if "Average" in df.columns:
        df = df[df["Average"] == "yes"]
    
    if in_squad_only:
        if "In_Squad" in df.columns:
            df = df[df["In_Squad"] == "Yes"]
        elif "In_squad" in df.columns:
            df = df[df["In_squad"] == "Yes"]
    
    return df.reset_index(drop=True)

# use all players (looks better on plots). Set to True if you only want current squad.
df_fw = load_position_sheet(EXCEL_PATH, "FW", in_squad_only=False)
df_mf = load_position_sheet(EXCEL_PATH, "MF", in_squad_only=False)
df_df = load_position_sheet(EXCEL_PATH, "DF", in_squad_only=False)

print("Forwards:", df_fw.shape)
print("Midfielders:", df_mf.shape)
print("Defenders:", df_df.shape)


Forwards: (14, 31)
Midfielders: (12, 39)
Defenders: (14, 39)


In [ ]:
# Forwards (FW)
fw_features = [
    "Gls", "Ast", "G+A", "Gls90", "Ast90",
    "MP", "Min", "PrgC", "PrgP", "PrgR"
]

# Midfielders (MF)
mf_features = [
    "Gls", "Ast", "G+A", "PrgC", "PrgP", "PrgR",
    "Tkl", "TklW", "Blocks", "Int", "Clr", "Err"
]

# Defenders (DF)
df_features = [
    "Tkl", "TklW", "Blocks", "Int", "Clr", "Err", "90s"
]


In [ ]:
def run_clustering(df, feature_cols, n_clusters=4, random_state=42, label=""):
    """
    KMeans + PCA(2D).
    If there are fewer players than n_clusters, it reduces n_clusters automatically.
    """
    df = df.copy()
    X = df[feature_cols].fillna(0.0).values
    n_samples = X.shape[0]

    if n_samples == 0:
        raise ValueError(f"No rows available for clustering for {label}.")

    effective_k = min(n_clusters, n_samples)
    if effective_k < n_clusters:
        print(f"⚠️ {label}: requested {n_clusters} clusters but only {n_samples} samples. Using {effective_k} clusters.")

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    kmeans = KMeans(
        n_clusters=effective_k,
        random_state=random_state,
        n_init=10
    )
    clusters = kmeans.fit_predict(X_scaled)
    df["Cluster_km"] = clusters

    pca = PCA(n_components=2, random_state=random_state)
    X_pca = pca.fit_transform(X_scaled)
    df["PCA1_km"] = X_pca[:, 0]
    df["PCA2_km"] = X_pca[:, 1]

    return df, scaler, kmeans, pca


In [ ]:
fw_clustered, fw_scaler, fw_km, fw_pca = run_clustering(df_fw, fw_features, n_clusters=4, label="FW")
mf_clustered, mf_scaler, mf_km, mf_pca = run_clustering(df_mf, mf_features, n_clusters=4, label="MF")
df_clustered, df_scaler, df_km, df_pca = run_clustering(df_df, df_features, n_clusters=4, label="DF")

fw_clustered.head()


,player,season,team,nation,born,MP,Starts,Min,90s,Gls,...,G+A-PK90,Multirole,Average,Cluster,PCA1,PCA2,In_Squad,Cluster_km,PCA1_km,PCA2_km
0,Alejandro Garnacho,2025.0,Chelsea,ARG,2004.0,25.755501,16.254512,1498.658714,16.629543,5.000618,...,0.547410,no,yes,3.0,-98.865586,1.790057,Yes,3,-0.981828,0.543837
1,Carney Chukwuemeka,2025.0,Chelsea,ENG,2003.0,11.499382,2.000000,290.484302,3.199827,0.500124,...,0.395098,yes,yes,1.0,-71.891381,9.416083,NaN,3,-2.545335,0.651512
2,Cole Palmer,2025.0,Chelsea,ENG,2002.0,35.000494,32.500865,2899.072152,32.250803,18.499135,...,0.684964,yes,yes,2.0,184.350090,114.169827,Yes,1,5.490043,0.343902
3,Estêvão Willian,2025.0,Chelsea,BRA,2007.0,31.000000,26.000000,2097.000000,23.300000,13.000000,...,0.820000,no,yes,2.0,236.865363,-29.990451,Yes,1,4.347120,0.848310
4,Jamie Gittens,2025.0,Chelsea,ENG,2004.0,19.005810,10.004326,875.586897,9.728739,3.001360,...,0.417588,no,yes,0.0,0.627709,-13.376685,Yes,3,-1.090289,0.045200


In [ ]:
def interactive_clusters_with_names(df, feature_cols, scaler, kmeans, pca, title):
    """
    Interactive PCA plot:
      - Points colored by cluster
      - Player name shown just above each point
      - Legend on the right side
      - Hover still shows full player info
    """
    df = df.copy()

    # make sure PCA coords exist
    if "PCA1_km" not in df.columns or "PCA2_km" not in df.columns:
        X = df[feature_cols].fillna(0.0).values
        X_scaled = scaler.transform(X)
        X_pca = pca.transform(X_scaled)
        df["PCA1_km"] = X_pca[:, 0]
        df["PCA2_km"] = X_pca[:, 1]

    # nicer cluster labels for legend
    df["cluster_label"] = "Cluster " + df["Cluster_km"].astype(int).astype(str)

    # choose hover columns
    base_cols = ["player", "team", "season", "Pos", "In_Squad", "In_squad"]
    hover_cols = []
    for c in base_cols + feature_cols:
        if c in df.columns and c not in hover_cols:
            hover_cols.append(c)
    hover_cols.append("cluster_label")

    hover_data = {c: True for c in hover_cols}

    # scatter with text labels
    fig = px.scatter(
        df,
        x="PCA1_km",
        y="PCA2_km",
        color="cluster_label",
        hover_name="player",
        hover_data=hover_data,
        title=title
    )

    # show markers + player name on top of each point
    fig.update_traces(
        mode="markers+text",
        text=df["player"],
        textposition="top center",
        marker=dict(size=11, line=dict(width=1)),
        showlegend=True
    )

    # legend on the right side
    fig.update_layout(
        template="plotly_white",
        legend_title_text="Cluster",
        xaxis_title="PCA1",
        yaxis_title="PCA2",
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.02
        )
    )

    fig.show()


In [ ]:
interactive_clusters_with_names(
    fw_clustered,
    feature_cols=fw_features,
    scaler=fw_scaler,
    kmeans=fw_km,
    pca=fw_pca,
    title="Chelsea Forwards – Clusters (PCA)"
)

interactive_clusters_with_names(
    mf_clustered,
    feature_cols=mf_features,
    scaler=mf_scaler,
    kmeans=mf_km,
    pca=mf_pca,
    title="Chelsea Midfielders – Clusters (PCA)"
)

interactive_clusters_with_names(
    df_clustered,
    feature_cols=df_features,
    scaler=df_scaler,
    kmeans=df_km,
    pca=df_pca,
    title="Chelsea Defenders – Clusters (PCA)"
)
